Calculate ensemble of sea level projections

In [1]:
import numpy as np
import xarray as xr

import sys
sys.path.append('../src')
import utils as ut

In [2]:
ds = xr.open_dataset('../data/dummy.nc')
ds = ds.isel(exp=slice(0,5))

In [17]:
for b,bmp in enumerate(ut.bmps):
    slr_nf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    slr_wf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    for s,ss in enumerate(ds.ssp.values):
        for e,es in enumerate(ds.esm.values):
            for i,ii in enumerate(ds.ism.values):
                TMP,IML,SLR = ut.iterate(ds,ism=ii,esm=es,ssp=ss,bmp=bmp)
                slr_nf[s,e,i,:] = np.sum(SLR[0,:,:],axis=1)
                slr_wf[s,e,i,:] = np.sum(SLR[-1,:,:],axis=1)
                print(f'{bmp:4s}',ss,f'{es:14s}',f'{ii:10s}',f'{slr_nf[s,e,i,-1]: .3f}',f'{slr_wf[s,e,i,-1]: .3f}')       
    #Save
    slr_nf2 = xr.DataArray(slr_nf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise without feedback'})
    slr_wf2 = xr.DataArray(slr_wf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise with feedback'})

    ds2 = xr.Dataset({'slr_nf':slr_nf2,'slr_wf':slr_wf2})
    ds2.to_netcdf(f'../data/ensemble_{bmp}.nc',mode='w')
    ds2.close()

lin  126 ACCESS-CM2     CISM_NCA    0.042  0.069
lin  126 ACCESS-CM2     FETI_ULB    0.061  0.120
lin  126 ACCESS-CM2     GRIS_LSC    0.014  0.019
lin  126 ACCESS-CM2     IMAU_UU     0.147  0.289
lin  126 ACCESS-CM2     ISSM_UCI    0.126  0.182
lin  126 ACCESS-CM2     MALI_DOE    0.066  0.115
lin  126 ACCESS-CM2     PISM_DMI    0.122  0.396
lin  126 ACCESS-CM2     PISM_PIK    0.059  0.090
lin  126 ACCESS-CM2     SICO_ILTS   0.132  0.259
lin  126 ACCESS-ESM1-5  CISM_NCA    0.002  0.008
lin  126 ACCESS-ESM1-5  FETI_ULB    0.002  0.012


KeyboardInterrupt: 